# Chicago Traffic Incidents Involving Cyclists/Pedestrians in Last 12 Months
### Prepared by: Michael Goodman
>### Data Source: Chicago Data Portal
>#### Vehicles API Documentation: https://dev.socrata.com/foundry/data.cityofchicago.org/68nd-jvt3
>#### People API Documentation: https://dev.socrata.com/foundry/data.cityofchicago.org/u6pd-qa9d

#### Libraries to Import

In [ ]:
#Imports
#use pip to install any needed libraries below
import pandas as pd
import requests
import json
import time
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from sodapy import Socrata
from cdp_secrets import *

#### Chicago Data Portal Connection

In [ ]:
#Constants
chicago_url="data.cityofchicago.org"
vehicle_api_root="68nd-jvt3"
people_api_root="u6pd-qa9d"
#crash_api_root=

#Chicago Data Portal Connection
cdp = Socrata(chicago_url,
                 token,
                 username,
                 password)


#### Create string variables for today and one year ago

In [ ]:
#Get dates for the last 12 months
today = datetime.now().strftime("%Y-%m-%d")

year_ago = (datetime.now() - relativedelta(years=1)).strftime("%Y-%m-%d")
print(today)
print(year_ago)

#### Queries for API Calls. Filters based on project specification

In [ ]:
#limit required for SoQL query, as we are filtering for one year limit is set high to capture all records
limit='1000000000'

#Filters use SQL structure, some where clauses use a different format. Refer to API Documentation for correct syntax
vehicle_filter= """Select crash_record_id,crash_date,make,model,vehicle_type 
                    where vehicle_type is not null and crash_date between '%s' and '%s'
                    limit %s"""%(year_ago,today,limit) 

people_filter= """Select person_id,crash_record_id,crash_date,person_type,age,sex,
                injury_classification,pedpedal_action,pedpedal_visibility,pedpedal_location
                Where CRASH_DATE between '%s' and '%s' and
                (person_type='PEDESTRIAN' or person_type='BICYCLE') limit %s"""% (year_ago,today,limit)

#crash_filter=



#### Create DataFrames from API Results

In [ ]:
#People Table Results
people_results = cdp.get(people_api_root, query=people_filter)

# Convert to pandas DataFrame
people_df = pd.DataFrame.from_records(people_results)

print(people_df)

In [ ]:
#Vehicle Table Results
vehicle_results = cdp.get(vehicle_api_root, query=vehicle_filter)

#Convert to pandas DataFrame
vehicle_df=pd.DataFrame.from_records(vehicle_results)

print(vehicle_df)

In [ ]:
#Crash Table Results
crash_results = cdp.get(crash_api_root, query=crash_filter)

#Convert to pandas DataFrame
crash_df=pd.DataFrame.from_records(crash_results)

print(crash_df)

#### Join DataFrames to create project Data Model

In [ ]:
combined_df=pd.merge(people_df,vehicle_df[['crash_record_id','vehicle_type']], on='crash_record_id', how='left')
print(combined_df)

In [ ]:
combined_df['crash_date']=pd.to_datetime(combined_df['crash_date'])

#### Print to Excel

In [ ]:
vehicle_df.to_excel('vehicle_draft.xlsx',index=True)